# **GP 2 - SCRAPING & API**

*Логирование:*

In [ ]:
import logging
from logging.handlers import RotatingFileHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.propagate = False

if logger.hasHandlers():
    logger.handlers.clear()

formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s:%(lineno)d] - %(message)s')

file_handler = RotatingFileHandler('parser.log', maxBytes=5_000_000, backupCount=3)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(console_handler)

2025-03-04 16:49:09,601 [DEBUG] [__main__:28] - Это сообщение DEBUG: должно выводиться только один раз.
2025-03-04 16:49:09,602 [INFO] [__main__:29] - Это сообщение INFO: тоже один раз, и в файл, и в консоль.


## API AVIASALES

*В качестве 1 из источника данных используем Aviasales - будем собирать данные путем сбора через API.*

In [ ]:
import requests
import pandas as pd
import time

logger.info("Получение данных от Aviasales, OpenWeatherMap ")

aviasales_api_key = '8a56da7bfb183a0451edc4fcc4234223'
origin_list = ['MOW', 'LED', 'AER', 'OVB', 'SVX', 'KZN', 'MRV', 'KGD', 'UFA', 'KJA', 'IKT', 'KUF', 'VVO']  # топ городов с аэропортами в России

final_offers = []

# "горящие" направления из Aviasales
logger.info("Получение 'горящих' направлений из Aviasales.")
for origin_key in origin_list:
    logger.debug(f"Отправляем запрос к API Aviasales для {origin_key}.")
    try:
        response = requests.get(
            f"https://api.travelpayouts.com/aviasales/v3/get_special_offers?origin={origin_key}&locale=en&token={aviasales_api_key}"
        )
        response.raise_for_status()  # выбросит исключение при плохом ответе
        response_data = response.json()

        if response_data.get('success') and 'data' in response_data:
            offers = response_data['data']
            logger.debug(f"Для {origin_key} получено предложений: {len(offers)}")

            for offer in offers:
                final_offers.append({
                    'Airline': offer.get('airline'),
                    'Airline Title': offer.get('airline_title'),
                    'Departure': offer.get('departure_at'),
                    'Destination': offer.get('destination'),
                    'Destination Name': offer.get('destination_name'),
                    'Duration': offer.get('duration'),
                    'Flight Number': offer.get('flight_number'),
                    'Price': offer.get('price'),
                    'Origin': offer.get('origin'),
                    'Origin Name': offer.get('origin_name'),
                    'Title': offer.get('title')
                })
        else:
            logger.warning(f"Нет данных для {origin_key}.")
    except requests.exceptions.RequestException as e:
        logger.exception(f"Ошибка при запросе к Aviasales для {origin_key}: {e}")

    # чтобы не перегружать API
    time.sleep(1)


df = pd.DataFrame(final_offers)
logger.info(f"Создан DataFrame df для горящих направлений размером {df.shape} .")

2025-03-04 16:49:27,496 [INFO] [__main__:5] - Получение данных от Aviasales, OpenWeatherMap 
2025-03-04 16:49:27,498 [INFO] [__main__:13] - Получение 'горящих' направлений из Aviasales.
2025-03-04 16:49:27,501 [DEBUG] [__main__:15] - Отправляем запрос к API Aviasales для MOW.
2025-03-04 16:49:27,848 [DEBUG] [__main__:25] - Для MOW получено предложений: 9
2025-03-04 16:49:28,849 [DEBUG] [__main__:15] - Отправляем запрос к API Aviasales для LED.
2025-03-04 16:49:29,089 [DEBUG] [__main__:25] - Для LED получено предложений: 9
2025-03-04 16:49:30,090 [DEBUG] [__main__:15] - Отправляем запрос к API Aviasales для AER.
2025-03-04 16:49:30,371 [DEBUG] [__main__:25] - Для AER получено предложений: 9
2025-03-04 16:49:31,372 [DEBUG] [__main__:15] - Отправляем запрос к API Aviasales для OVB.
2025-03-04 16:49:31,509 [DEBUG] [__main__:25] - Для OVB получено предложений: 9
2025-03-04 16:49:32,510 [DEBUG] [__main__:15] - Отправляем запрос к API Aviasales для SVX.
2025-03-04 16:49:32,811 [DEBUG] [__main

In [ ]:
df

,Airline,Airline Title,Departure,Destination,Destination Name,Duration,Flight Number,Price,Origin,Origin Name,Title
0,DP,Pobeda,2025-03-15T09:30:00+03:00,ERF,Erfurt,1680,839,28136,MOW,Moscow,Flight deals from Moscow to Erfurt
1,DP,Pobeda,2025-03-06T08:30:00+03:00,GLA,Glasgow,795,839,13892,MOW,Moscow,Flight deals from Moscow to Glasgow
2,DP,Pobeda,2025-03-08T08:30:00+03:00,CMF,Chambery,2085,839,33596,MOW,Moscow,Flight deals from Moscow to Chambery
3,TK,Turkish Airlines,2025-03-28T07:50:00+03:00,YMQ,Montreal,1105,737,46216,MOW,Moscow,Flight deals from Moscow to Montreal
4,MU,China Eastern Airlines,2025-03-10T21:05:00+03:00,NSN,Nelson,2950,204,91087,MOW,Moscow,Flight deals from Moscow to Nelson
...,...,...,...,...,...,...,...,...,...,...,...
112,MU,China Eastern Airlines,2025-03-14T19:05:00+10:00,KMG,Kunming,475,8299,10854,VVO,Vladivostok,Flight deals from Vladivostok to Kunming
113,CA,Air China,2025-04-03T18:10:00+10:00,SGN,Ho Chi Minh City,570,812,20112,VVO,Vladivostok,Flight deals from Vladivostok to Ho Chi Minh City
114,IO,IrAero,2025-04-08T07:45:00+10:00,NHA,Nha Trang,890,6404,47932,VVO,Vladivostok,Flight deals from Vladivostok to Nha Trang
115,CA,Air China,2025-04-03T18:10:00+10:00,BJS,Beijing,160,812,4582,VVO,Vladivostok,Flight deals from Vladivostok to Beijing


In [ ]:
# по каждому направлению билеты
logger.info("Получение деталей билетов по каждому направлению.")

all_results = []

pairs = df[['Origin', 'Destination']].drop_duplicates()
logger.debug(f"Уникальных пар Origin-Destination: {pairs.shape[0]}")

for index, row in pairs.iterrows():
    origin = row['Origin']
    destination = row['Destination']

    url = 'https://api.travelpayouts.com/aviasales/v3/prices_for_dates'
    params = {
        'origin': origin,
        'destination': destination,
        'limit': 1000,
        'token': aviasales_api_key
    }

    logger.debug(f"Запрос к API  для пары {origin}-{destination}")
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        json_response = response.json()

        if 'data' in json_response and json_response['data']:
            flights = json_response['data']
            logger.debug(f"Для пары {origin}-{destination} получено билетов: {len(flights)}")

            for flight in flights:
                all_results.append({
                    'origin': flight.get('origin'),
                    'destination': flight.get('destination'),
                    'price': flight.get('price'),
                    'airline': flight.get('airline'),
                    'flight_number': flight.get('flight_number'),
                    'departure_at': flight.get('departure_at'),
                    'transfers': flight.get('transfers'),
                    'duration': flight.get('duration'),
                    'link': flight.get('link')
                })
        else:
            logger.warning(f'Нет данных  для направления {origin} → {destination}')
    except requests.exceptions.RequestException as e:
        logger.exception(f'Ошибка при запросе к Aviasales для {origin} → {destination}: {e}')


    time.sleep(1.5)

df_flights = pd.DataFrame(all_results)
logger.info(f"Создан DataFrame df_flights размера {df_flights.shape} (детали билетов).")

2025-03-04 16:49:57,791 [INFO] [__main__:2] - Получение деталей билетов по каждому направлению.
2025-03-04 16:49:57,797 [DEBUG] [__main__:7] - Уникальных пар Origin-Destination: 117
2025-03-04 16:49:57,799 [DEBUG] [__main__:21] - Запрос к API  для пары MOW-ERF
2025-03-04 16:49:58,065 [DEBUG] [__main__:29] - Для пары MOW-ERF получено билетов: 14
2025-03-04 16:49:59,566 [DEBUG] [__main__:21] - Запрос к API  для пары MOW-GLA
2025-03-04 16:49:59,867 [DEBUG] [__main__:29] - Для пары MOW-GLA получено билетов: 35
2025-03-04 16:50:01,368 [DEBUG] [__main__:21] - Запрос к API  для пары MOW-CMF
2025-03-04 16:50:01,771 [DEBUG] [__main__:29] - Для пары MOW-CMF получено билетов: 7
2025-03-04 16:50:03,273 [DEBUG] [__main__:21] - Запрос к API  для пары MOW-YMQ
2025-03-04 16:50:03,484 [DEBUG] [__main__:29] - Для пары MOW-YMQ получено билетов: 120
2025-03-04 16:50:04,986 [DEBUG] [__main__:21] - Запрос к API  для пары MOW-NSN
2025-03-04 16:50:05,250 [DEBUG] [__main__:29] - Для пары MOW-NSN получено билет

## API OpenWeather

В качестве 1 из источника данных используем OpenWeather - будем собирать данные путем сбора через API.

In [ ]:
#собираем погоду с OpenWeatherMap
logger.info("Запрос данных о погоде через OpenWeatherMap")
openweather_api_key = '15c5ac1c39b750c1e4770d7ab6970cea'
geocode_url = 'http://api.openweathermap.org/geo/1.0/direct'
weather_url = 'https://api.openweathermap.org/data/2.5/weather'

destination_to_weather = {}
unique_destinations = df['Destination Name'].unique()
logger.debug(f"Уникальных Destination Name: {len(unique_destinations)}")

for destination_name in unique_destinations:
    params = {
        'q': destination_name.split()[0],
        'limit': 1,
        'appid': openweather_api_key
    }
    try:
        openweather_response = requests.get(geocode_url, params=params)
        openweather_response.raise_for_status()
        geocode_data = openweather_response.json()

        if len(geocode_data) > 0:
            lat = geocode_data[0].get('lat')
            lon = geocode_data[0].get('lon')
            logger.debug(f"Координаты для {destination_name}: lat={lat}, lon={lon}")

            weather_params = {
                'lat': lat,
                'lon': lon,
                'appid': openweather_api_key,
                'units': 'metric'
            }
            weather_response = requests.get(weather_url, params=weather_params)
            weather_response.raise_for_status()
            weather_data = weather_response.json()

            main_weather = weather_data['weather'][0]['main']
            temperature = weather_data['main']['temp']
            destination_to_weather[destination_name] = {
                'Weather': main_weather,
                'Temperature': temperature
            }
        else:
            logger.warning(f"Не удалось найти координаты для '{destination_name}'")

    except requests.exceptions.RequestException as e:
        logger.exception(f"Ошибка при получении данных о погоде для {destination_name}: {e}")
    except KeyError as e:
        logger.error(f"Некорректная структура ответа OpenWeatherMap для {destination_name}: {e}")

# добавляем данные о погоде в DataFrame
df['Weather'] = df['Destination Name'].map(
    lambda name: destination_to_weather[name]['Weather'] if name in destination_to_weather else None
)
df['Temperature'] = df['Destination Name'].map(
    lambda name: destination_to_weather[name]['Temperature'] if name in destination_to_weather else None
)
logger.info("Информация о погоде добавлена в DataFrame -  df.")

2025-03-04 16:54:31,055 [INFO] [__main__:2] - Запрос данных о погоде через OpenWeatherMap
2025-03-04 16:54:31,058 [DEBUG] [__main__:9] - Уникальных Destination Name: 81
2025-03-04 16:54:31,230 [DEBUG] [__main__:25] - Координаты для Erfurt: lat=50.9777974, lon=11.0287364
2025-03-04 16:54:31,464 [DEBUG] [__main__:25] - Координаты для Glasgow: lat=55.8609825, lon=-4.2488787
2025-03-04 16:54:31,676 [DEBUG] [__main__:25] - Координаты для Chambery: lat=45.5662672, lon=5.9203636
2025-03-04 16:54:31,886 [DEBUG] [__main__:25] - Координаты для Montreal: lat=45.5031824, lon=-73.5698065
2025-03-04 16:54:32,094 [DEBUG] [__main__:25] - Координаты для Nelson: lat=-41.2710849, lon=173.2836756
2025-03-04 16:54:32,296 [DEBUG] [__main__:25] - Координаты для Djerba: lat=33.77339055, lon=10.885904100766005
2025-03-04 16:54:32,508 [DEBUG] [__main__:25] - Координаты для Brisbane: lat=-27.4689682, lon=153.0234991
2025-03-04 16:54:32,709 [DEBUG] [__main__:25] - Координаты для Toulon: lat=43.1257311, lon=5.9304

## Парсинг WikiVoyage (selenium + bs4)

*В качестве 1 из источника данных используем WikiVoyage - будем собирать данные путем сбора через Selenium и BeautifulSoup.*

Разбираемся с установкой одинаковых версий chrome driver и сhrome, для того чтобы selenium заработал.

In [ ]:
!apt-get update
!apt-get install -y wget gnupg

# скачиваем для линукса (колаб использует линукс) гугл, скачиваем для селениума
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable
!pip install selenium webdriver-manager


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 1s (5,087 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

Непосредственно сам парсинг:

In [ ]:
#Парсинг Wikivoyage
logger.info("Парсинг страниц Wikivoyage ( с помощью selenium).")

import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

def parse_destination(destination_name):
    # парсим страницы Wikivoyage для конкретного destination_name, где возвращаем списки типов мест и наименований
    logger.debug(f"Для {destination_name}.")
    url = f'https://en.wikivoyage.org/wiki/{destination_name}#See'
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, 'html.parser')
        titles = [link['title'] for link in soup.find_all('a') if link.has_attr('title')]

        place_types = []
        places = []
        recording = False

        start_marker = 'Edit section: See'
        end_marker = 'Edit section: Do'

        for title in titles:
            if title == start_marker:
                recording = True
            elif title == end_marker:
                break
            elif recording:
                if 'Edit section: ' in title:
                    place_types.append(title.replace('Edit section: ', ''))
                else:
                    places.append(title)

        logger.debug(f"Найдено {len(place_types)} типов мест и {len(places)} мест для {destination_name}.")
        return place_types, places

    except Exception as e:
        logger.exception(f"Ошибка при парсинге  для {destination_name}: {e}")
        return [], []

if 'Place Types' not in df.columns:
    df['Place Types'] = pd.Series([], dtype='object')
if 'Places' not in df.columns:
    df['Places'] = pd.Series([], dtype='object')

for index, row in df.iterrows():
    destination_name = row['Destination Name']
    place_types, places = parse_destination(destination_name)
    df.at[index, 'Place Types'] = [place.split(' (')[0] for place in place_types]
    df.at[index, 'Places'] = [place.split(' (')[0] for place in places]

logger.info("Информация о местах с Wikivoyage добавлена в DataFrame df.")

2025-03-04 17:07:11,821 [INFO] [__main__:2] - Парсинг страниц Wikivoyage ( с помощью selenium).
2025-03-04 17:07:11,825 [DEBUG] [__main__:17] - Для Erfurt.
2025-03-04 17:07:18,466 [DEBUG] [__main__:47] - Найдено 1 типов мест и 20 мест для Erfurt.
2025-03-04 17:07:18,467 [DEBUG] [__main__:17] - Для Glasgow.
2025-03-04 17:07:26,998 [DEBUG] [__main__:47] - Найдено 6 типов мест и 84 мест для Glasgow.
2025-03-04 17:07:27,000 [DEBUG] [__main__:17] - Для Chambery.
2025-03-04 17:07:33,531 [DEBUG] [__main__:47] - Найдено 0 типов мест и 1 мест для Chambery.
2025-03-04 17:07:33,533 [DEBUG] [__main__:17] - Для Montreal.
2025-03-04 17:07:40,740 [DEBUG] [__main__:47] - Найдено 3 типов мест и 12 мест для Montreal.
2025-03-04 17:07:40,741 [DEBUG] [__main__:17] - Для Nelson.
2025-03-04 17:07:46,990 [DEBUG] [__main__:47] - Найдено 0 типов мест и 0 мест для Nelson.
2025-03-04 17:07:46,991 [DEBUG] [__main__:17] - Для Djerba.
2025-03-04 17:07:53,615 [DEBUG] [__main__:47] - Найдено 0 типов мест и 20 мест дл

In [ ]:
# мерджим датасеты  df_flights и df
logger.info("Объединение данных о билетах и данных о направлениях/погоде/местах.")
data = pd.merge(df_flights, df, left_on='destination', right_on='Destination', how='left')
logger.info(f"Собран финальный датасет data размера {data.shape} для исследования")

2025-03-04 17:28:08,406 [INFO] [__main__:2] - Объединение данных о билетах и данных о направлениях/погоде/местах.
2025-03-04 17:28:08,460 [INFO] [__main__:4] - Собран финальный датасет data размера (23743, 24) для исследования


In [ ]:
data

,origin,destination,price,airline,flight_number,departure_at,transfers,duration,link,Airline,...,Duration,Flight Number,Price,Origin,Origin Name,Title,Weather,Temperature,Place Types,Places
0,MOW,ERF,22553,DP,839,2025-03-08T08:30:00+03:00,1,1740,/search/MOW0803ERF1?t=DP1741422600174151980000...,DP,...,1680.0,839,28136.0,MOW,Moscow,Flight deals from Moscow to Erfurt,Clouds,8.93,[Jewish heritage],"[Erfurt Cathedral, Erfurt Cathedral on Wikiped..."
1,MOW,ERF,22553,DP,839,2025-03-08T08:30:00+03:00,1,1740,/search/MOW0803ERF1?t=DP1741422600174151980000...,DP,...,1475.0,6708,25906.0,LED,Saint Petersburg,Flight deals from Saint Petersburg to Erfurt,Clouds,8.93,[Jewish heritage],"[Erfurt Cathedral, Erfurt Cathedral on Wikiped..."
2,MOW,ERF,28136,DP,839,2025-03-15T09:30:00+03:00,1,1680,/search/MOW1503ERF1?t=DP1742031000174212460000...,DP,...,1680.0,839,28136.0,MOW,Moscow,Flight deals from Moscow to Erfurt,Clouds,8.93,[Jewish heritage],"[Erfurt Cathedral, Erfurt Cathedral on Wikiped..."
3,MOW,ERF,28136,DP,839,2025-03-15T09:30:00+03:00,1,1680,/search/MOW1503ERF1?t=DP1742031000174212460000...,DP,...,1475.0,6708,25906.0,LED,Saint Petersburg,Flight deals from Saint Petersburg to Erfurt,Clouds,8.93,[Jewish heritage],"[Erfurt Cathedral, Erfurt Cathedral on Wikiped..."
4,MOW,ERF,31610,2S,012,2025-03-16T00:50:00+03:00,1,760,/search/MOW1603ERF1?t=2S1742086200174212460000...,DP,...,1680.0,839,28136.0,MOW,Moscow,Flight deals from Moscow to Erfurt,Clouds,8.93,[Jewish heritage],"[Erfurt Cathedral, Erfurt Cathedral on Wikiped..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23738,VVO,DPS,116719,S7,5204,2025-11-19T08:50:00+10:00,4,3690,/search/VVO1911DPS1?t=S71763542200176375640000...,HO,...,830.0,6245,39034.0,VVO,Vladivostok,Flight deals from Vladivostok to Denpasar (Bali),Clouds,27.82,[],[]
23739,VVO,DPS,120878,S7,5204,2026-01-02T08:50:00+10:00,4,3065,/search/VVO0201DPS1?t=S71767343800176752050000...,HO,...,830.0,6245,39034.0,VVO,Vladivostok,Flight deals from Vladivostok to Denpasar (Bali),Clouds,27.82,[],[]
23740,VVO,DPS,145107,S7,6404,2026-02-01T07:45:00+10:00,4,3755,/search/VVO0102DPS1?t=MH1769931900177015000000...,HO,...,830.0,6245,39034.0,VVO,Vladivostok,Flight deals from Vladivostok to Denpasar (Bali),Clouds,27.82,[],[]
23741,VVO,DPS,171216,S7,6404,2025-11-01T07:45:00+10:00,2,3615,/search/VVO0111DPS1?t=S71761983100176219280000...,HO,...,830.0,6245,39034.0,VVO,Vladivostok,Flight deals from Vladivostok to Denpasar (Bali),Clouds,27.82,[],[]


In [ ]:
data.to_csv("data.csv", index=False, encoding="utf-8")


Мы собрали датасет и он готов к дальнейшему исследованию на EDA!